<a href="https://colab.research.google.com/github/ikanx101/G-Colab/blob/main/Optimasi_dua_produk_dua_plant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [134]:
#system("sudo apt-get install libglpk-dev")
#install.packages("ROI")
#install.packages("ompr")
#install.packages("ompr.roi")
#install.packages("ROI.plugin.glpk")

In [135]:
library(dplyr)
library(ompr)
library(ompr.roi)
library(ROI.plugin.glpk)
library(ggplot2)

rm(list=ls())

In [136]:
problem = 
data.frame(
  keterangan = c("Cars","Trucks","Man Hour"),
  plant_1 = c("2 man days","5 man days","150 man days"),
  plant_2 = c("3 man days","3 man days","140 man days"),
  demand_perweeks = c(50,30,""),
  profit = c(150,250,"")
)

problem

keterangan,plant_1,plant_2,demand_perweeks,profit
<chr>,<chr>,<chr>,<chr>,<chr>
Cars,2 man days,3 man days,50,150
Trucks,5 man days,3 man days,30,250
Man Hour,150 man days,140 man days,,


In [137]:
# misalkan:
# i : banyak produk, i=1,2
# j : banyak plant, j=1,2

# x[i,j] banyaknya produk i yang diproduksi di plant j
# profit[i] : profit pada produk i

In [138]:
profit = c(150,250)

model = 
  MIPModel() %>%
  add_variable(x[i,j],
               i = 1:2,
               j = 1:2,
               type = "integer",
               lb = 0) %>%
  set_objective(sum_expr(x[i,j] * profit[i], i = 1:2,j = 1:2),
                "max"
               ) %>%
  add_constraint(2 * x[1,1] + 5 * x[2,1] <= 150) %>%
  add_constraint(3 * x[1,2] + 3 * x[2,2] <= 140) %>%
  add_constraint(x[1,1] + x[1,2] >= 50) %>%
  add_constraint(x[2,1] + x[2,2] >= 30)
model


Mixed integer linear optimization problem
Variables:
  Continuous: 0 
  Integer: 4 
  Binary: 0 
Model sense: maximize 
Constraints: 4 

In [139]:
result <- solve_model(model, with_ROI(solver = "glpk", verbose = TRUE))


<SOLVER MSG>  ----
GLPK Simplex Optimizer, v4.65
4 rows, 4 columns, 8 non-zeros
      0: obj =  -0.000000000e+00 inf =   8.000e+01 (2)
      3: obj =   1.500000000e+04 inf =   0.000e+00 (0)
*     5: obj =   2.291666667e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Integer Optimizer, v4.65
4 rows, 4 columns, 8 non-zeros
4 integer variables, none of which are binary
Integer optimization begins...
Long-step dual simplex will be used
+     5: mip =     not found yet <=              +inf        (1; 0)
+     7: >>>>>   2.275000000e+04 <=   2.280000000e+04   0.2% (2; 0)
+     7: mip =   2.275000000e+04 <=     tree is empty   0.0% (0; 3)
INTEGER OPTIMAL SOLUTION FOUND
<!SOLVER MSG> ----


In [140]:
result$solution

x[1,1] x[2,1] x[1,2] x[2,2] 
    75      0      0     46

In [141]:
result$objective_value

[1] 22750